#### Heatmapに関するコード
ファイルの名称について
- acと遺伝子の長さを対応させる
    - clinvar_ac2seqnum.json
- acと変異数を対応させる
    - clinvar_ac2varCount

In [ ]:
#ACとタンパク質の長さを対応させる
import json
from collections import defaultdict

clinvar_ac2length = defaultdict(list)
acs =[]
seq = ""

data = open("../uniprot_human.dat")
for line in data:
    if line.startswith("AC"):
        info = line.rstrip().split()
        for b in info[1:]:
            acs.append(b.replace(";",""))  
    if line.startswith("SQ"):
        seq=line.rstrip().split()[2]
    elif line.startswith("//"):
        clinvar_ac2length[acs[0]]=seq
        acs = []
        seq = ""

with open('../clinvar_ac2length.json', 'w') as f:
    json.dump(clinvar_ac2length, f, indent=4)

In [ ]:
#変異数カウント
import json
from collections import defaultdict

json_open = open("../clinvar_1ac.json","r")
Clinvar_1ac_human_classfied = json.load(json_open)

Clinvar_varCount = defaultdict(int)

for data in Clinvar_1ac_human_classfied:
    for data2 in Clinvar_1ac_human_classfied[data]:
        Clinvar_varCount[data]+=1
        
with open('../clinvar_ac2varCount.json', 'w') as f:
    json.dump(Clinvar_varCount, f, indent=4)

In [ ]:
#クラス
#subcel: 局在先が入ったリスト
#ac2locs: acと局在先を対応させるjson
#clinvar
#ac2varCount
#ac2length

def get_heatmap(subcel,ac2locs,clinvar,ac2varCount,ac2length):
    import json
    import math
    from collections import defaultdict

    disease = {}
    polymorphism = {}
    for s in subcel:
        disease[s] = defaultdict(int)
        polymorphism[s] = defaultdict(int)

    aa = ['Ile', 'Val', 'Leu', 'Phe', 'Cys', 'Met', 'Ala', 'Gly', 'Thr', 'Ser', 'Trp', 'Tyr', 'Pro', 'His', 'Glu', 'Gln', 'Asp', 'Asn', 'Lys', 'Arg']
    counter = {}
    for s in subcel:
        counter[s] = defaultdict(int)

    numCounter = defaultdict(int)
    otherslist = set()

    for ac in clinvar:
        length = int(ac2length[ac])
        varCount = ac2varCount[ac]
        for data in clinvar[ac]:
            if ac in ac2locs.keys():
                for compartment in ac2locs[ac]:
                    numCounter[compartment] += 1
                    counter[compartment][data["Variant"]["before"]+data["Variant"]["after"]] += 1/length/varCount
                    if data["Clinical_significance"]== "Disease":
                        disease[compartment][data["Variant"]["before"]+data["Variant"]["after"]] += 1/length/varCount 
                    elif data["Clinical_significance"] == "Polymorphism":
                        polymorphism[compartment][data["Variant"]["before"]+data["Variant"]["after"]] += 1/length/varCount

    wo_localize = {}
    for b in aa:
        wo_localize[b] = {}
        for a in aa:
            var = b + a
            d = 0 if var not in disease['All'] else disease['All'][var]
            p = 0 if var not in polymorphism['All'] else polymorphism['All'][var]
            if d + p != 0:
                wo_localize[b][a] = d/(d+p)
            else:
                wo_localize[b][a] = "-"


    for s in subcel:
        print("{}\t{}".format(s,"\t".join(aa)))
        for b in aa:
            print("{}\t".format(b),end="")
            for a in aa:
                var = b + a
                d = 0 if var not in disease[s] else disease[s][var]
                p = 0 if var not in polymorphism[s] else polymorphism[s][var]

                if d + p != 0 and wo_localize[b][a] != '-':
                    try:
                        try:
                            #常用対数とる
                            # print("{}\t".format(math.log((d / (d + p)) /(pd_Clinvar_classified["All"][b][a]))),end="")
                            # jsonから読み込んだ値ではなく上で計算した値を利用する
                            print("{}\t".format(math.log((d / (d + p)) /(wo_localize[b][a]))),end="")
                            #取らない
                            #print("{}\t".format((d / (d + p)),end=""))
                        except ValueError:
                            print("-\t",end="")
                    except ZeroDivisionError:
                        print("-\t",end="")
                else:
                    print("-\t",end="")
            print()
        print()

In [ ]:
#例
json_open = open('../ac2locs_classified_multi.json', 'r')
ac2locs = json.load(json_open)

json_open = open('../clinvar_1ac.json', 'r')
clinvar = json.load(json_open)

json_open = open("../clinvar_ac2varCount.json","r")
ac2varCount = json.load(json_open)

json_open = open('../clinvar_ac2length.json', 'r')
ac2length = json.load(json_open)

subcel = ["All",'Mitochondrion', 'Others', 'Cytoplasm,Mitochondrion','Cytoplasm,Nucleus', 'Nucleus', 'Lysosome', 'Membrane', 'Secreted', 'Cytoplasm', "ER,Golgi",'NoData']
subcel = []

In [ ]:
get_heatmap(subcel,ac2locs,clinvar,ac2varCount,ac2length)